### Ein 1-2-3-Labyrinth

```python
    0    1    2    3    4    5
0[['S', ' ', ' ', ' ', ' ', ' '],
1 ['X', 'X', 'X', 'X', 'X', 'X'],
2 ['X', ' ', 'X', ' ', ' ', 'X'],
3 ['X', ' ', 'X', 'X', 'X', 'X'],
4 ['X', ' ', 'X', ' ', ' ', 'X'],
5 ['X', ' ', ' ', ' ', ' ', 'Z'],
6 ['X', ' ', ' ', ' ', ' ', 'X'],
7 ['X', 'X', 'X', 'X', 'X', 'X']]
 ```

Ziel ist es, vom Start ('S') ins Ziel ('Z') zu gelangen.
Dabei sind folgende Regeln einzuhalten:

- Man darf sich nur auf den markierten Feldern 'S', 'X' und 'Z' bewegen.
- Im 1. Zug bewegt man sich 1 Feld weit, im 2. Zug 2, im 3. Zug 3, im 4. Zug wieder 1 Feld weit, u.s.w.  
- W&auml;hrend eines Zuges darf man sich nur in **eine Richtung** bewegen.

### Weg zum Ziel suchen
- Wir nummeriern die markierten Felder von links nach rechts und von oben nach unten:  
  S ist Feld 0, Z ist Feld 19, das Feld in Reihe 2, Spalte 0 ist Feld 7, das Feld in Reihe 3, Spalte 0 ist Feld 10.
- F&uuml;r jedes  der markierten 28 Feld gibt es 3 Zust&auml;nde, je nach dem, ob man sich gerade
  1, 2 oder 3 Felder bewegen darf. Das macht $28\cdot 3 = 84$ Zust&auml;nde $S:=\{0,1,\ldots,83\}$.
  Dabei interpretieren wir Zustand $n\in S$ als "Spieler ist auf Feld
  `n //  3` und muss sich `n%3 + 1` Felder bewegen". Im Zustand 4 ist der Spieler auf dem 1. Feld und muss sich 2 Felder bewegen.

  
- Ein Dictionary `delta` legt die legalen Z&uuml;ge fest. F&uuml;r jeden Zustand geben wir die Menge der erreichbaren Zust&auml;nde an.
  ```python
    0: {4},
    1: {23},
    2: {30},
    3: {1, 7, 22},
    4: {11, 32},
    5: {12, 45},
    ...
   }
  ```
<img src="../../images/labyrith_states.svg">

In [1]:
from dict_helpers import peek


labyrinth = [
  ['S', ' ', ' ', ' ', ' ', ' '],
  ['X', 'X', 'X', 'X', 'X', 'X'],
  ['X', ' ', 'X', ' ', ' ', 'X'],
  ['X', ' ', 'X', 'X', 'X', 'X'],
  ['X', ' ', 'X', ' ', ' ', 'X'],
  ['X', ' ', ' ', ' ', ' ', 'Z'],
  ['X', ' ', ' ', ' ', ' ', 'X'],
  ['X', 'X', 'X', 'X', 'X', 'X'],
]

In [2]:
NCOLS = 6
NROWS = 8

positions = []
for y in range(NROWS):
    for x in range(NCOLS):
        if labyrinth[y][x] != ' ':
            positions.append((x, y))
positions[:4], len(positions)

([(0, 0), (0, 1), (1, 1), (2, 1)], 28)

In [ ]:
field_pos = {k: v for k, v in enumerate(positions)}
pos_field = {v: k for k, v in field_pos.items()}
peek(field_pos, 2), peek(pos_field, 2)

In [ ]:
nstates = len(field_pos) * 3
nstates

In [ ]:
def is_position(pos):
    x, y = pos
    return (0 <= x < NCOLS and
            0 <= y < NROWS and
            labyrinth[y][x] != ' ')

In [ ]:
def get_path(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    dx = abs(x2-x1)
    dy = abs(y2-y1)
    
    if dx == 0:
        y = min(y1, y2)
        return [(x1, y+i) for i in range(dy+1)]
        
    if dy == 0:
        x = min(x1, x2)
        return [(x+i, y1) for i in range(dx+1)]

In [ ]:
def is_connected(pos1, pos2):
    path = get_path(pos1, pos2)
    return all(is_position(pos) for pos in path)

In [ ]:
def options(field, n):
    pos = field_pos[field]
    x, y = pos
    positions = [(x+n, y),
                 (x-n, y),
                 (x, y+n),
                 (x, y-n),
                 ]
    fields = [pos_field[p] for p in positions if is_connected(pos, p)]
    return fields

In [ ]:
def reachable_states(state):
    field = state//3
    r = state%3 
    fields = options(field, r+1)
    r = (r+1) % 3
    return set(3*field + r for field in fields)

In [ ]:
reachable_states(74)

In [ ]:
delta = {}
for state in range(nstates):
    delta[state] = reachable_states(state)
delta

In [62]:
def explore(delta, state_path, frontier):
    new_frontier = set()
    for state in frontier:
        new_states = (s for s in delta[state] if s not in state_path)
        for s in new_states:
            new_path = state_path[state].copy()
            new_path.append(s)
            state_path[s] = new_path
            new_frontier.add(s)
    return new_frontier

In [63]:
start = 0
frontier = {start}
frontiers = [frontier]
state_path = {start: [start]}

In [64]:
frontier = explore(delta, state_path, frontier)
state_path, frontier

({0: [0], 4: [0, 4]}, {4})

In [65]:
S = 0
frontier = {S}
frontiers = [frontier]
state_path = {S: [S]}

while frontier:
    frontier = explore(delta, state_path, frontier)

In [66]:
Z = 19
[s for s in state_path if s // 3 == Z]

[57]

In [68]:
path = state_path[57]
path

[0, 4, 32, 60, 67, 56, 21, 31, 5, 12, 10, 35, 42, 52, 29, 57]

In [72]:
[field_pos[state//3] for state in path]

[(0, 0),
 (0, 1),
 (0, 3),
 (0, 6),
 (0, 7),
 (0, 5),
 (0, 2),
 (0, 3),
 (0, 1),
 (3, 1),
 (2, 1),
 (2, 3),
 (5, 3),
 (5, 4),
 (5, 2),
 (5, 5)]